In [1]:
print("ok")

ok


In [ ]:
from langchian import promptTemplate 
from langchain.chains import RetrievalQA
from langchain.chains import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
import Pinecone
from langchian.document_loders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import promptTemplate
from langchain.llms import CTransformers

In [ ]:
PINECONE_API_KEY = "PINECONE_KEY"
PINECONE_API_ENV = "PINECONE_ENV"
## https://medical-bot-f67g429.svc.aped-4627-b74a.pinecone.io

In [ ]:
# Extract data from the pdf
def load_pdf(Data):
    loader = DirectoryLoader(Data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
Extracted_data = load_pdf("Data/")

In [ ]:
# Create text chunks
def text_split(extraxcted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overloap = 20)
    text_chunks = text_splitter.split_documents(Extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(Extracted_data)
print("length of my chunks:", len(text_chunks))

In [ ]:
# download embedding model
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [ ]:
embeddings = download_hugging_face_embedding():
# it will download from hugging face itself

In [ ]:
#embeddings

In [ ]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

In [ ]:
#query_result

In [ ]:
# Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-bot"

# Creating Embedding for each of the Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
# if we already have an index we can load it like this

docsearch = Pinecone.from_exisitng_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query,k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to naswer the user's question.
if you don't know the answer, just say that you don't know,don't try to make up an answer.

context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result['result'])